# Manim Animation

In [1]:
from manim import (
    BLUE,
    LEFT,
    RIGHT,
    UP,
    DOWN,
    Circle,
    Create,
    Dot,
    GrowFromCenter,
    Homotopy,
    Line,
    MoveAlongPath,
    ParametricFunction,
    Rotating,
    Scene,
    Transform,
    linear,
)

import numpy as np
import scipy.interpolate

Manim Community v0.17.3

In [9]:
%%manim --disable_caching -v ERROR GrowDotToCircle

class GrowDotToCircle(Scene):
    def construct(self):
        circle = Circle()
        circle.set_fill(BLUE, opacity=0.5)

        self.add(circle)
        self.wait(2)
        self.remove(circle)
        self.wait(2)
        self.play(Create(circle))

In [10]:
%%manim PointMovingOnShapes

class PointMovingOnShapes(Scene):
    def construct(self):
        circle = Circle(radius=1, color=BLUE)
        dot = Dot()
        dot2 = dot.copy().shift(RIGHT)
        self.add(dot)

        line = Line([3, 0, 0], [5, 0, 0])
        self.add(line)

        self.play(GrowFromCenter(circle))
        self.play(Transform(dot, dot2))
        self.play(MoveAlongPath(dot, circle), run_time=2, rate_func=linear)
        self.play(Rotating(dot, about_point=[2, 0, 0]), run_time=1.5)
        self.wait()

In [11]:
def animate_pong(ts, positions, bounds):
    """ Create a Pong game animator. """

    class AnimatePong(Scene):
        def _bounding_line(self, bounds, start, end):
            start = [bounds[start[0]], bounds[start[1]], 0.0]
            end = [bounds[end[0]], bounds[end[1]], 0.0]
            return Line(start=start, end=end, color=BLUE)
        
        def _bounding_lines(self, bounds):
            return [
                self._bounding_line(bounds, (0, 2), (0, 3)),
                self._bounding_line(bounds, (0, 3), (1, 3)),
                self._bounding_line(bounds, (1, 3), (1, 2)),
                self._bounding_line(bounds, (1, 2), (0, 2)),
            ]
        
        def construct(self):
            t_range = [ts[0], ts[-1]]
            spline_t = scipy.interpolate.make_interp_spline(ts, positions, k=1)
            path = ParametricFunction(spline_t, t_range=t_range)
            
            # bounding box
            self.add(*self._bounding_lines(bounds))

            # pong ball
            dot = Dot()
            self.add(dot)
            self.play(MoveAlongPath(dot, path))
    
    return AnimatePong

In [97]:
%%manim --disable_caching AnimatePong

ts = np.linspace(0, 2 * np.pi, 100)
positions = np.array([2 * np.sin(ts)**2, 2 * np.cos(ts)**2, 0.0 * ts]).T
bounds = [2, 0, 0, 2]  # top, bottom, left, right

AnimatePong = animate_pong(ts, positions, bounds)